In [50]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model = 'deepseek-r1-distill-llama-70b'
)

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001BCDAFC67B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BCDB112000>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
print(llm.invoke("hi, how are you?").content)

<think>

</think>

Hi! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How are you doing? 😊


In [5]:
print(llm.invoke("How is the today's weather in Kolkata, India?").content)

<think>

</think>

As of my last update in July 2024, I don't have real-time information. Please enable the Internet to get the current weather information.


In [10]:
import operator
from typing import List
from langgraph.graph.message import add_messages
from pydantic import BaseModel , Field
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

In [43]:
msg = MessagesState()
print(type(msg))

<class 'dict'>


In [ ]:
def call_llm(state: MessagesState) -> dict:
    messages = state['messages']
    response = llm.invoke(messages)
    return {"messages": [response]}

In [27]:
messages = [HumanMessage(content="""
                        What is NATO, and when was it formed? Does NATO still hold strategic significance even after the collapse of the USSR and 
                        considering Russia's relatively weaker economy compared to the U.S. and Europe? Some argue that NATO now serves more as a 
                        display of Western power and may even pose a threat to global peace. What are your thoughts on this perspective — especially given 
                        the unpredictability of nations that feel cornered or powerless? You probably know who I'm referring to.
                        """)]

state: MessagesState = {"messages": messages}
print(type(state))

response = call_llm(state)

<class 'dict'>


In [36]:
print(response['messages'][0].content)

<think>
Okay, so I'm trying to understand what NATO is and why it's still important today. I know NATO stands for North Atlantic Treaty Organization, but I'm not entirely sure when it was formed or what its original purpose was. I think it was created after World War II, maybe in the late 1940s or early 1950s. The main idea was to form a military alliance between Western countries to counter the Soviet Union during the Cold War. But now that the Soviet Union has collapsed, I'm wondering why NATO still exists.

I remember hearing that NATO expanded after the USSR fell, including more Eastern European countries. That must have made some people, especially in Russia, feel threatened or surrounded. So, does NATO still serve a strategic purpose, or is it just a show of Western power? Some argue that it's more about displaying strength now, which could actually make things more tense, especially with Russia.

The user also mentioned that NATO might pose a threat to global peace, especially i

In [37]:
state = {"messages":["hi hello how are you?"]}
response = call_llm(state)

In [41]:
print(response['messages'][0].content)

<think>

</think>

Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How are *you* doing? 😊


In [ ]:
from langchain_core.tools import tool
import requests

@tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    
    api_key = os.getenv("OPENWEATHER_API_KEY")
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code != 200:
        return f"Failed to get weather: {response.text}"
    
    data = response.json()
    print(data)
    
    temp = data["main"]["temp"]
    desc = data["weather"][0]["description"]
    
    return f"The weather in {city} is {desc} with a temperature of {temp}°C."


In [45]:
get_weather.description

'Get current weather for a city.'

In [49]:
get_weather.args

{'city': {'title': 'City', 'type': 'string'}}

In [59]:
get_weather.invoke("bangalore")

{'coord': {'lon': 77.6033, 'lat': 12.9762}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 26.56, 'feels_like': 26.56, 'temp_min': 26.56, 'temp_max': 26.56, 'pressure': 1010, 'humidity': 73, 'sea_level': 1010, 'grnd_level': 912}, 'visibility': 10000, 'wind': {'speed': 6.55, 'deg': 266, 'gust': 12.7}, 'clouds': {'all': 100}, 'dt': 1750874389, 'sys': {'country': 'IN', 'sunrise': 1750811130, 'sunset': 1750857524}, 'timezone': 19800, 'id': 1277333, 'name': 'Bengaluru', 'cod': 200}


'The weather in bangalore is overcast clouds with a temperature of 26.56°C.'

{'coord': {'lon': 77.6033, 'lat': 12.9762}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 26.56, 'feels_like': 26.56, 'temp_min': 26.56, 'temp_max': 26.56, 'pressure': 1010, 'humidity': 73, 'sea_level': 1010, 'grnd_level': 912}, 'visibility': 10000, 'wind': {'speed': 6.55, 'deg': 266, 'gust': 12.7}, 'clouds': {'all': 100}, 'dt': 1750874389, 'sys': {'country': 'IN', 'sunrise': 1750811130, 'sunset': 1750857524}, 'timezone': 19800, 'id': 1277333, 'name': 'Bengaluru', 'cod': 200}


In [60]:
get_weather.invoke("kolkata")

{'coord': {'lon': 88.3697, 'lat': 22.5697}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 28.71, 'feels_like': 34.76, 'temp_min': 28.71, 'temp_max': 28.71, 'pressure': 1001, 'humidity': 84, 'sea_level': 1001, 'grnd_level': 1000}, 'visibility': 10000, 'wind': {'speed': 4.61, 'deg': 128, 'gust': 8.55}, 'clouds': {'all': 100}, 'dt': 1750874302, 'sys': {'country': 'IN', 'sunrise': 1750807418, 'sunset': 1750856068}, 'timezone': 19800, 'id': 1275004, 'name': 'Kolkata', 'cod': 200}


'The weather in kolkata is overcast clouds with a temperature of 28.71°C.'

{'coord': {'lon': 88.3697, 'lat': 22.5697}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 28.71, 'feels_like': 34.76, 'temp_min': 28.71, 'temp_max': 28.71, 'pressure': 1001, 'humidity': 84, 'sea_level': 1001, 'grnd_level': 1000}, 'visibility': 10000, 'wind': {'speed': 4.61, 'deg': 128, 'gust': 8.55}, 'clouds': {'all': 100}, 'dt': 1750874302, 'sys': {'country': 'IN', 'sunrise': 1750807418, 'sunset': 1750856068}, 'timezone': 19800, 'id': 1275004, 'name': 'Kolkata', 'cod': 200}


In [61]:
get_weather.invoke("moscow")

{'coord': {'lon': 37.6156, 'lat': 55.7522}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 15.87, 'feels_like': 15.55, 'temp_min': 15.87, 'temp_max': 15.87, 'pressure': 997, 'humidity': 78, 'sea_level': 997, 'grnd_level': 979}, 'visibility': 10000, 'wind': {'speed': 4.41, 'deg': 225, 'gust': 10.35}, 'clouds': {'all': 26}, 'dt': 1750880254, 'sys': {'country': 'RU', 'sunrise': 1750812353, 'sunset': 1750875498}, 'timezone': 10800, 'id': 524901, 'name': 'Moscow', 'cod': 200}


'The weather in moscow is scattered clouds with a temperature of 15.87°C.'

{'coord': {'lon': 37.6156, 'lat': 55.7522}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 15.87, 'feels_like': 15.55, 'temp_min': 15.87, 'temp_max': 15.87, 'pressure': 997, 'humidity': 78, 'sea_level': 997, 'grnd_level': 979}, 'visibility': 10000, 'wind': {'speed': 4.41, 'deg': 225, 'gust': 10.35}, 'clouds': {'all': 26}, 'dt': 1750880254, 'sys': {'country': 'RU', 'sunrise': 1750812353, 'sunset': 1750875498}, 'timezone': 10800, 'id': 524901, 'name': 'Moscow', 'cod': 200}
